<!-- <h2>Title</h2> -->

---
<h1> 
    National Income Dynamics Study-Coronavirus Rapid Mobile Survey (NIDS-CRAM)<br>
    Wave 1: February - April 2020<br><br>
    The Effect of COVID-19 on the Labour and Welfare of South Africans
</h1>

--- 


<h1> 
Part 2 - Rough Draft
</h1>

--- 


<h2>Group Members</h2>
Dino Anastasopoulos: 1900661 - Contributed towards the Demographics section. <br>
Brenton Budler: 1827655 - Contributed towards the Labour section. <br>
Philani Mpofu: 1848751 - Contributed towards the Welfare section. <br>

--- 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Group-Members" data-toc-modified-id="Group-Members-1">Group Members</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2">Imports</a></span></li><li><span><a href="#Read-data" data-toc-modified-id="Read-data-3">Read data</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-4">Model</a></span><ul class="toc-item"><li><span><a href="#Drop-all-nans" data-toc-modified-id="Drop-all-nans-4.1">Drop all nans</a></span></li><li><span><a href="#Now-this-is-the-dataframe-of-all-people-who-kept-their-jobs,-and-have-usable-data" data-toc-modified-id="Now-this-is-the-dataframe-of-all-people-who-kept-their-jobs,-and-have-usable-data-4.2">Now this is the dataframe of all people who kept their jobs, and have usable data</a></span></li><li><span><a href="#Calculate-and-store-salary-difference" data-toc-modified-id="Calculate-and-store-salary-difference-4.3">Calculate and store salary difference</a></span></li><li><span><a href="#Remove-outliers" data-toc-modified-id="Remove-outliers-4.4">Remove outliers</a></span></li><li><span><a href="#Determine-if-salary-changed-and-create-new-variable-for-this" data-toc-modified-id="Determine-if-salary-changed-and-create-new-variable-for-this-4.5">Determine if salary changed and create new variable for this</a></span></li></ul></li><li><span><a href="#Check-value-counts-of-salary-change" data-toc-modified-id="Check-value-counts-of-salary-change-5">Check value counts of salary change</a></span></li><li><span><a href="#Check-details-of-work-type" data-toc-modified-id="Check-details-of-work-type-6">Check details of work type</a></span><ul class="toc-item"><li><span><a href="#Remove-all-nonsensical-dara-(not-applicable-and-unknown)-and-jobs-with-too-few-occurences" data-toc-modified-id="Remove-all-nonsensical-dara-(not-applicable-and-unknown)-and-jobs-with-too-few-occurences-6.1">Remove all nonsensical dara (not applicable and unknown) and jobs with too few occurences</a></span></li></ul></li><li><span><a href="#Check-details-of-province" data-toc-modified-id="Check-details-of-province-7">Check details of province</a></span><ul class="toc-item"><li><span><a href="#Encode-some-variables-and-get-dummies" data-toc-modified-id="Encode-some-variables-and-get-dummies-7.1">Encode some variables and get dummies</a></span></li></ul></li></ul></div>

## Imports

In [168]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

import matplotlib.cm as cm

from matplotlib.colors import Normalize

import statsmodels.formula.api as smf
import statsmodels.api as sm

from IPython.display import display_html,display, Markdown

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler, LabelEncoder




np.set_printoptions(suppress=True, precision=7)
pd.set_option("display.max_columns",55)
print("Successfully imported all")





Successfully imported all


## Read data

In [261]:
dem = pd.read_csv("Demographics.csv",)
lab = pd.read_csv("Labour.csv")

df = pd.concat([dem,lab],axis=1)

## Model

**Looking at what factors determined a change in a persons salary**

In [262]:
dftemp = pd.concat([dem,lab],axis=1)

In [263]:
display(dftemp)
dftemp.shape

,age,gender,race,highest_grade,tertiary_edu,province_current,province_moved,province_before,province_during,labour_in_feb,work_days_feb,work_hours_feb,take_home_pay_feb,labour_in_apr,work_days_apr,work_hours_apr,take_home_pay_apr,lost_labour,kept_labour,return_to_work,usual_work
0,39.0,Man,White,Grade 9,No,Gauteng,No,NaN,NaN,1,NaN,NaN,180.0,0,0.0,0.0,0.0,1,0,0,Unknown
1,72.0,Woman,Asian/Indian,Grade 0,NaN,KwaZulu-Natal,No,NaN,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
2,30.0,Woman,African/Black,Grade 10,Yes,Gauteng,No,NaN,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
3,48.0,Woman,African/Black,Grade 10,No,Gauteng,No,NaN,NaN,1,3.0,5.0,900.0,0,0.0,0.0,0.0,1,0,1,Elementary occupations
4,49.0,Woman,African/Black,Grade 9,No,Gauteng,No,NaN,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7068,22.0,Man,African/Black,Grade 11,No,Northern Cape,No,NaN,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
7069,24.0,Woman,Coloured,Grade 10,No,Western Cape,No,NaN,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
7070,36.0,Woman,African/Black,NTC 3,No,North West,No,NaN,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
7071,20.0,Man,African/Black,Grade 12,No,North West,Yes,Gauteng,Gauteng,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed


(7073, 21)

In [265]:
def fix_race(race):
    if race == "African/Black": return "Black"
    elif race == "Asian/Indian": return "AsInd"
    else: return race
    
    
def fix_province(provinceCurrent, provinceMoved, provinceBefore):
    if provinceMoved == "No":
        province = provinceCurrent
    else:
        province = provinceBefore
        
    return province

def fix_punc(province):
    prov = province.replace(" ", "_")
    prov = prov.replace("-", "_")

    return prov


    
def fix_work(work):
    new_work = work.replace(" ", "_")
    new_work = new_work.replace("-", "_")
    new_work = new_work.replace(",", "")
    return new_work



dftemp["race"] = np.vectorize(fix_race)(dftemp["race"])


#Fix province and puncuation
dftemp["province_before"] = np.vectorize(fix_province)(dftemp["province_current"],dftemp["province_moved"],dftemp["province_before"])

dftemp["province_before"] = np.vectorize(fix_punc)(dftemp["province_before"])

#Fix puncuation in usual work
dftemp["usual_work"] = np.vectorize(fix_work)(dftemp["usual_work"])


display(dftemp)

,age,gender,race,highest_grade,tertiary_edu,province_current,province_moved,province_before,province_during,labour_in_feb,work_days_feb,work_hours_feb,take_home_pay_feb,labour_in_apr,work_days_apr,work_hours_apr,take_home_pay_apr,lost_labour,kept_labour,return_to_work,usual_work
0,39.0,Man,White,Grade 9,No,Gauteng,No,Gauteng,NaN,1,NaN,NaN,180.0,0,0.0,0.0,0.0,1,0,0,Unknown
1,72.0,Woman,AsInd,Grade 0,NaN,KwaZulu-Natal,No,KwaZulu_Natal,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not_Employed
2,30.0,Woman,Black,Grade 10,Yes,Gauteng,No,Gauteng,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not_Employed
3,48.0,Woman,Black,Grade 10,No,Gauteng,No,Gauteng,NaN,1,3.0,5.0,900.0,0,0.0,0.0,0.0,1,0,1,Elementary_occupations
4,49.0,Woman,Black,Grade 9,No,Gauteng,No,Gauteng,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not_Employed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7068,22.0,Man,Black,Grade 11,No,Northern Cape,No,Northern_Cape,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not_Employed
7069,24.0,Woman,Coloured,Grade 10,No,Western Cape,No,Western_Cape,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not_Employed
7070,36.0,Woman,Black,NTC 3,No,North West,No,North_West,NaN,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not_Employed
7071,20.0,Man,Black,Grade 12,No,North West,Yes,Gauteng,Gauteng,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not_Employed


In [319]:
# df_main = dftemp[["age","gender","race","highest_grade","province_current", "tertiary_edu","take_home_pay_feb", "take_home_pay_apr","kept_labour","usual_work"]]




df_main = dftemp[["age","gender","race","highest_grade", "tertiary_edu","province_before","take_home_pay_feb", "take_home_pay_apr","kept_labour","usual_work"]]


### Drop all nans

In [320]:
display(df_main)
df_main_dropped_nans = df_main.dropna()
display(df_main_dropped_nans)

,age,gender,race,highest_grade,tertiary_edu,province_before,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work
0,39.0,Man,White,Grade 9,No,Gauteng,180.0,0.0,0,Unknown
1,72.0,Woman,AsInd,Grade 0,NaN,KwaZulu_Natal,0.0,0.0,0,Not_Employed
2,30.0,Woman,Black,Grade 10,Yes,Gauteng,0.0,0.0,0,Not_Employed
3,48.0,Woman,Black,Grade 10,No,Gauteng,900.0,0.0,0,Elementary_occupations
4,49.0,Woman,Black,Grade 9,No,Gauteng,0.0,0.0,0,Not_Employed
...,...,...,...,...,...,...,...,...,...,...
7068,22.0,Man,Black,Grade 11,No,Northern_Cape,0.0,0.0,0,Not_Employed
7069,24.0,Woman,Coloured,Grade 10,No,Western_Cape,0.0,0.0,0,Not_Employed
7070,36.0,Woman,Black,NTC 3,No,North_West,0.0,0.0,0,Not_Employed
7071,20.0,Man,Black,Grade 12,No,Gauteng,0.0,0.0,0,Not_Employed


,age,gender,race,highest_grade,tertiary_edu,province_before,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work
0,39.0,Man,White,Grade 9,No,Gauteng,180.0,0.0,0,Unknown
2,30.0,Woman,Black,Grade 10,Yes,Gauteng,0.0,0.0,0,Not_Employed
3,48.0,Woman,Black,Grade 10,No,Gauteng,900.0,0.0,0,Elementary_occupations
4,49.0,Woman,Black,Grade 9,No,Gauteng,0.0,0.0,0,Not_Employed
5,34.0,Man,Black,Grade 9,No,Gauteng,0.0,0.0,0,Not_Employed
...,...,...,...,...,...,...,...,...,...,...
7068,22.0,Man,Black,Grade 11,No,Northern_Cape,0.0,0.0,0,Not_Employed
7069,24.0,Woman,Coloured,Grade 10,No,Western_Cape,0.0,0.0,0,Not_Employed
7070,36.0,Woman,Black,NTC 3,No,North_West,0.0,0.0,0,Not_Employed
7071,20.0,Man,Black,Grade 12,No,Gauteng,0.0,0.0,0,Not_Employed


### Now this is the dataframe of all people who kept their jobs, and have usable data

In [321]:
# df_main_kept_labour = df_main_diff[(df_main_diff['kept_labour']==1) & (df_main_diff['salary_change']!="None")]


df_main_kept_labour = df_main_dropped_nans[(df_main_dropped_nans['kept_labour']==1)]

print(list(df_main_kept_labour))

display(df_main_kept_labour)


['age', 'gender', 'race', 'highest_grade', 'tertiary_edu', 'province_before', 'take_home_pay_feb', 'take_home_pay_apr', 'kept_labour', 'usual_work']


,age,gender,race,highest_grade,tertiary_edu,province_before,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work
15,52.0,Woman,Coloured,Grade 12,No,Gauteng,25000.0,25000.0,1,Technicians_and_associate_professionals
20,40.0,Woman,Black,Grade 11,No,Gauteng,3500.0,3500.0,1,Technicians_and_associate_professionals
41,37.0,Woman,Black,Grade 12,Yes,Gauteng,4600.0,4600.0,1,Professionals
44,45.0,Woman,Black,Grade 12,Yes,Gauteng,4000.0,4200.0,1,Service_and_sales_workers
45,39.0,Woman,Black,Grade 11,Yes,Gauteng,3500.0,7200.0,1,Professionals
...,...,...,...,...,...,...,...,...,...,...
7001,43.0,Woman,Black,Grade 9,No,Eastern_Cape,3200.0,3200.0,1,Elementary_occupations
7016,29.0,Man,Black,Grade 10,No,North_West,4400.0,4500.0,1,Service_and_sales_workers
7018,30.0,Woman,Black,Grade 12,Yes,Free_State,24000.0,12000.0,1,Professionals
7021,27.0,Woman,Black,Grade 9,No,Limpopo,1500.0,2800.0,1,Unknown


### Calculate and store salary difference

In [322]:

diff = df_main_kept_labour['take_home_pay_apr'] - df_main_kept_labour['take_home_pay_feb']

display(diff)

perc = (diff/df_main_kept_labour['take_home_pay_feb'])* 100

perc = np.nan_to_num(perc)
# display(perc)

df_main_diff = df_main_kept_labour.copy()

df_main_diff['take_home_pay_difference'] = perc

15          0.0
20          0.0
41          0.0
44        200.0
45       3700.0
         ...   
7001        0.0
7016      100.0
7018   -12000.0
7021     1300.0
7038        0.0
Length: 706, dtype: float64

### Remove outliers

In [513]:
print("Min percentage change before: ",min(df_main_diff['take_home_pay_difference']))
print("Max percentage change before: ", max(df_main_diff['take_home_pay_difference']))

# Removing Income outliers
df_no_outliers = df_main_diff[df_main_diff["take_home_pay_difference"]<np.percentile(a = df_main_diff["take_home_pay_difference"], q =98)]


print("\n\n\nMin percentage change after: ",min(df_no_outliers['take_home_pay_difference']))
print("Max percentage change after: ",max(df_no_outliers['take_home_pay_difference']))
display(df_no_outliers)

Min percentage change before:  -100.0
Max percentage change before:  1.7976931348623157e+308



Min percentage change after:  -100.0
Max percentage change after:  135.29411764705884


,age,gender,race,highest_grade,tertiary_edu,province_before,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work,take_home_pay_difference
15,52.0,Woman,Coloured,Grade 12,No,Gauteng,25000.0,25000.0,1,Technicians_and_associate_professionals,0.000000
20,40.0,Woman,Black,Grade 11,No,Gauteng,3500.0,3500.0,1,Technicians_and_associate_professionals,0.000000
41,37.0,Woman,Black,Grade 12,Yes,Gauteng,4600.0,4600.0,1,Professionals,0.000000
44,45.0,Woman,Black,Grade 12,Yes,Gauteng,4000.0,4200.0,1,Service_and_sales_workers,5.000000
45,39.0,Woman,Black,Grade 11,Yes,Gauteng,3500.0,7200.0,1,Professionals,105.714286
...,...,...,...,...,...,...,...,...,...,...,...
7001,43.0,Woman,Black,Grade 9,No,Eastern_Cape,3200.0,3200.0,1,Elementary_occupations,0.000000
7016,29.0,Man,Black,Grade 10,No,North_West,4400.0,4500.0,1,Service_and_sales_workers,2.272727
7018,30.0,Woman,Black,Grade 12,Yes,Free_State,24000.0,12000.0,1,Professionals,-50.000000
7021,27.0,Woman,Black,Grade 9,No,Limpopo,1500.0,2800.0,1,Unknown,86.666667


In [514]:
df_newvars = df_no_outliers.copy()

def check_youth(age):
    if age<25: return 1
    else: return 0 

def check_male(gender): 
    if gender == "Man": return 1
    else: return 0
    
def check_female(gender): 
    if gender == "Woman": return 1
    else: return 0
    

def check_matric(highest_grade):
    if highest_grade == "Grade 12": return 1
    else: return 0

def check_tertedu(tertiary_edu):
    if tertiary_edu == "Yes" : return 1 
    else: return 0 
    
def check_work(work):
    new_work = work.replace(" ", "_")
    new_work = new_work.replace("-", "_")
    new_work = new_work.replace(",", "")
    return new_work


def check_white(race):
    if race == "White": return 1
    else: return 0
    
def check_black(race):
    if race == "Black": return 1
    else: return 0

    
XYZ = pd.DataFrame()
XYZ["Age"] = df_newvars["age"]
XYZ["Youth"] = np.vectorize(check_youth)(df_newvars["age"])

XYZ["Race"] = np.vectorize(check_race)(df_newvars["race"])



XYZ["Black"] = np.vectorize(check_black)(df_newvars["race"])
XYZ["White"] = np.vectorize(check_white)(df_newvars["race"])


# XYZ["Race"] = df_newvars["race"]


XYZ["Male"] = np.vectorize(check_male)(df_newvars["gender"])

XYZ["Female"] = np.vectorize(check_female)(df_newvars["gender"])


XYZ["Matric"] = np.vectorize(check_matric)(df_newvars["highest_grade"])
XYZ["Tertiary"] = np.vectorize(check_tertedu)(df_newvars["tertiary_edu"])


XYZ["Province"] = df_newvars["province_before"]


XYZ["Usual_Work"] = np.vectorize(check_work)(df_newvars['usual_work']) 

display(XYZ)

,Age,Youth,Race,Black,White,Male,Female,Matric,Tertiary,Province,Usual_Work
15,52.0,0,Coloured,0,0,0,1,1,0,Gauteng,Technicians_and_associate_professionals
20,40.0,0,Black,1,0,0,1,0,0,Gauteng,Technicians_and_associate_professionals
41,37.0,0,Black,1,0,0,1,1,1,Gauteng,Professionals
44,45.0,0,Black,1,0,0,1,1,1,Gauteng,Service_and_sales_workers
45,39.0,0,Black,1,0,0,1,0,1,Gauteng,Professionals
...,...,...,...,...,...,...,...,...,...,...,...
7001,43.0,0,Black,1,0,0,1,0,0,Eastern_Cape,Elementary_occupations
7016,29.0,0,Black,1,0,1,0,0,0,North_West,Service_and_sales_workers
7018,30.0,0,Black,1,0,0,1,1,1,Free_State,Professionals
7021,27.0,0,Black,1,0,0,1,0,0,Limpopo,Unknown


### Determine if salary changed and create new variable for this

In [515]:
def salary_change(diff):
    if diff==0:
        return "unchanged"
    else:
        return "changed"
    

def salary_diff(diff):
    if diff==0:
        return "unchanged"
    elif diff>0:
        return "increased"
    elif diff<0:
        return "decreased"
    
    
def salary_decreased(diff):
    if diff<0:
        return "decreased"
    else:
        return "did_not_decrease"
    
def salary_increased(diff):
    if diff>0:
        return "increased"
    else:
        return "did_not_increased"
    
    
def salary_inc_dec(diff):
    if diff<0:
        return "decreased"
    elif diff>0:
        return "increased"
    else:
        return "none"


XYZ["Pay_Feb"] = df_newvars["take_home_pay_feb"]
XYZ["Pay_Apr"] = df_newvars["take_home_pay_apr"]

XYZ["Pay_Diff"] = df_newvars['take_home_pay_difference']
XYZ["Salary_Change"] = np.vectorize(salary_change)(df_newvars['take_home_pay_difference']) 
XYZ["Salary_Diff"] = np.vectorize(salary_diff)(df_newvars['take_home_pay_difference']) 
XYZ["Salary_Decreased"] = np.vectorize(salary_decreased)(df_newvars['take_home_pay_difference']) 
XYZ["Salary_Increased"] = np.vectorize(salary_increased)(df_newvars['take_home_pay_difference']) 

XYZ["Salary_IncDec"] = np.vectorize(salary_inc_dec)(df_newvars['take_home_pay_difference']) 

#Normalize salaries
# XYZ["Pay_Feb"] = XYZ["Pay_Feb"] = (XYZ["Pay_Feb"]-XYZ["Pay_Feb"].min())/ (XYZ["Pay_Feb"].max()-XYZ["Pay_Feb"].min())

# XYZ["Pay_Apr"] = XYZ["Pay_Apr"] = (XYZ["Pay_Apr"]-XYZ["Pay_Apr"].min())/ (XYZ["Pay_Apr"].max()-XYZ["Pay_Apr"].min())

# XYZ["Pay_Diff"] = XYZ["Pay_Diff"] = (XYZ["Pay_Diff"]-XYZ["Pay_Diff"].min())/ (XYZ["Pay_Diff"].max()-XYZ["Pay_Diff"].min())

display(XYZ)

,Age,Youth,Race,Black,White,Male,Female,Matric,Tertiary,Province,Usual_Work,Pay_Feb,Pay_Apr,Pay_Diff,Salary_Change,Salary_Diff,Salary_Decreased,Salary_Increased,Salary_IncDec
15,52.0,0,Coloured,0,0,0,1,1,0,Gauteng,Technicians_and_associate_professionals,25000.0,25000.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
20,40.0,0,Black,1,0,0,1,0,0,Gauteng,Technicians_and_associate_professionals,3500.0,3500.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
41,37.0,0,Black,1,0,0,1,1,1,Gauteng,Professionals,4600.0,4600.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
44,45.0,0,Black,1,0,0,1,1,1,Gauteng,Service_and_sales_workers,4000.0,4200.0,5.000000,changed,increased,did_not_decrease,increased,increased
45,39.0,0,Black,1,0,0,1,0,1,Gauteng,Professionals,3500.0,7200.0,105.714286,changed,increased,did_not_decrease,increased,increased
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7001,43.0,0,Black,1,0,0,1,0,0,Eastern_Cape,Elementary_occupations,3200.0,3200.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
7016,29.0,0,Black,1,0,1,0,0,0,North_West,Service_and_sales_workers,4400.0,4500.0,2.272727,changed,increased,did_not_decrease,increased,increased
7018,30.0,0,Black,1,0,0,1,1,1,Free_State,Professionals,24000.0,12000.0,-50.000000,changed,decreased,decreased,did_not_increased,decreased
7021,27.0,0,Black,1,0,0,1,0,0,Limpopo,Unknown,1500.0,2800.0,86.666667,changed,increased,did_not_decrease,increased,increased


## Check value counts of salary change

In [516]:
display(Markdown("## Salary Change Value Counts"))
print(XYZ["Salary_Change"].value_counts())

## Salary Change Value Counts

unchanged    423
changed      268
Name: Salary_Change, dtype: int64


## Check details of work type

In [517]:
display(Markdown("## Work type with Salary_Change == \"decreased\""))
print(XYZ[XYZ["Salary_Change"]=="decreased"]["Usual_Work"].value_counts())

display(Markdown("## Work type with Salary_Change == \"increased\""))
print(XYZ[XYZ["Salary_Change"]=="increased"]["Usual_Work"].value_counts())

display(Markdown("## Work type with Salary_Change == \"unchanged\""))
print(XYZ[XYZ["Salary_Change"]=="unchanged"]["Usual_Work"].value_counts())

display(Markdown("## Work type with Salary_Change == \"changed\""))
print(XYZ[XYZ["Salary_Change"]=="changed"]["Usual_Work"].value_counts())

display(Markdown("## Work type with Salary_Change == \"did_not_decrease\""))
print(XYZ[XYZ["Salary_Change"]=="did_not_decrease"]["Usual_Work"].value_counts())

display(Markdown("## Work Type Value Counts"))
print(XYZ["Usual_Work"].value_counts())

## Work type with Salary_Change == "decreased"

Series([], Name: Usual_Work, dtype: int64)


## Work type with Salary_Change == "increased"

Series([], Name: Usual_Work, dtype: int64)


## Work type with Salary_Change == "unchanged"

...

Service_and_sales_workers                            78
Professionals                                        40
Elementary_occupations                               40
Clerical_support_workers                             31
Plant_and_machine_operators_and_assemblers           20
Technicians_and_associate_professionals              18
Craft_and_related_trades_workers                     15
Managers                                             15
Unknown                                               5
Skilled_agricultural_forestry_and_fishery_workers     2
Not_Employed                                          1
Not_applicable                                        1
Self_Employed                                         1
Armed_forces_occupations                              1
Name: Usual_Work, dtype: int64


## Work type with Salary_Change == "did_not_decrease"

Series([], Name: Usual_Work, dtype: int64)


## Work Type Value Counts

Service_and_sales_workers                            146
Professionals                                        133
Elementary_occupations                               112
Clerical_support_workers                              83
Technicians_and_associate_professionals               72
Managers                                              49
Plant_and_machine_operators_and_assemblers            45
Craft_and_related_trades_workers                      30
Unknown                                               13
Skilled_agricultural_forestry_and_fishery_workers      3
Self_Employed                                          2
Armed_forces_occupations                               1
Not_applicable                                         1
Not_Employed                                           1
Name: Usual_Work, dtype: int64


### Remove all nonsensical dara (not applicable and unknown) and jobs with too few occurences

In [518]:
XYZ = XYZ[XYZ.groupby("Usual_Work")["Usual_Work"].transform(len) > 15]
XYZ["Usual_Work"].value_counts()

Service_and_sales_workers                     146
Professionals                                 133
Elementary_occupations                        112
Clerical_support_workers                       83
Technicians_and_associate_professionals        72
Managers                                       49
Plant_and_machine_operators_and_assemblers     45
Craft_and_related_trades_workers               30
Name: Usual_Work, dtype: int64

## Check details of province

In [519]:
display(Markdown("## Province with Salary_Change == \"decreased\""))
print(XYZ[XYZ["Salary_Change"]=="decreased"]["Province"].value_counts())

display(Markdown("## Province with Salary_Change == \"increased\""))
print(XYZ[XYZ["Salary_Change"]=="increased"]["Province"].value_counts())

display(Markdown("## Province with Salary_Change == \"unchanged\""))
print(XYZ[XYZ["Salary_Change"]=="unchanged"]["Province"].value_counts())

display(Markdown("## Province with Salary_Change == \"changed\""))
print(XYZ[XYZ["Salary_Change"]=="changed"]["Province"].value_counts())

display(Markdown("## Province with Salary_Change == \"did_not_decrease\""))
print(XYZ[XYZ["Salary_Change"]=="did_not_decrease"]["Province"].value_counts())

display(Markdown("## Province Value Counts"))
print(XYZ["Province"].value_counts())

## Province with Salary_Change == "decreased"

Series([], Name: Province, dtype: int64)


## Province with Salary_Change == "increased"

Series([], Name: Province, dtype: int64)


## Province with Salary_Change == "unchanged"

...

Gauteng          69
KwaZulu_Natal    47
Mpumalanga       36
Free_State       25
Eastern_Cape     20
Northern_Cape    17
Western_Cape     16
Limpopo          16
North_West       11
Name: Province, dtype: int64


## Province with Salary_Change == "did_not_decrease"

Series([], Name: Province, dtype: int64)


## Province Value Counts

Gauteng          156
KwaZulu_Natal    124
Mpumalanga        78
Western_Cape      70
Eastern_Cape      64
Northern_Cape     55
Free_State        49
Limpopo           42
North_West        31
nan                1
Name: Province, dtype: int64


In [520]:
XYZ = XYZ[XYZ.groupby("Province")["Province"].transform(len) > 1]
XYZ["Province"].value_counts()

Gauteng          156
KwaZulu_Natal    124
Mpumalanga        78
Western_Cape      70
Eastern_Cape      64
Northern_Cape     55
Free_State        49
Limpopo           42
North_West        31
Name: Province, dtype: int64

In [521]:
XYZ["Race"].value_counts()
XYZ = XYZ[XYZ["Race"] != "nan"]

XYZ

,Age,Youth,Race,Black,White,Male,Female,Matric,Tertiary,Province,Usual_Work,Pay_Feb,Pay_Apr,Pay_Diff,Salary_Change,Salary_Diff,Salary_Decreased,Salary_Increased,Salary_IncDec
15,52.0,0,Coloured,0,0,0,1,1,0,Gauteng,Technicians_and_associate_professionals,25000.0,25000.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
20,40.0,0,Black,1,0,0,1,0,0,Gauteng,Technicians_and_associate_professionals,3500.0,3500.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
41,37.0,0,Black,1,0,0,1,1,1,Gauteng,Professionals,4600.0,4600.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
44,45.0,0,Black,1,0,0,1,1,1,Gauteng,Service_and_sales_workers,4000.0,4200.0,5.000000,changed,increased,did_not_decrease,increased,increased
45,39.0,0,Black,1,0,0,1,0,1,Gauteng,Professionals,3500.0,7200.0,105.714286,changed,increased,did_not_decrease,increased,increased
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6982,35.0,0,Black,1,0,0,1,1,1,Mpumalanga,Service_and_sales_workers,5200.0,5300.0,1.923077,changed,increased,did_not_decrease,increased,increased
7001,43.0,0,Black,1,0,0,1,0,0,Eastern_Cape,Elementary_occupations,3200.0,3200.0,0.000000,unchanged,unchanged,did_not_decrease,did_not_increased,none
7016,29.0,0,Black,1,0,1,0,0,0,North_West,Service_and_sales_workers,4400.0,4500.0,2.272727,changed,increased,did_not_decrease,increased,increased
7018,30.0,0,Black,1,0,0,1,1,1,Free_State,Professionals,24000.0,12000.0,-50.000000,changed,decreased,decreased,did_not_increased,decreased


### Encode some variables and get dummies

In [530]:
# df_dummies = pd.get_dummies(df_encoded, columns=["race","highest_grade","province_current", "usual_work"], drop_first=True)

LE = LabelEncoder()

# df_dummies = XYZ.copy()

df_dummies = pd.get_dummies(XYZ, columns=["Race","Usual_Work", "Province"])

# df_dummies = pd.get_dummies(XYZ, columns=["Race","Province","Usual_Work"], drop_first=True)

# df_dummies["Race"] = LE.fit_transform(df_dummies["Race"])
# df_dummies["Usual_Work"] = LE.fit_transform(df_dummies["Usual_Work"])
df_dummies["Salary_Change"] = 1 - LE.fit_transform(df_dummies["Salary_Change"])
df_dummies["Salary_Decreased"] = 1 - LE.fit_transform(df_dummies["Salary_Decreased"])
df_dummies["Salary_Increased"] = LE.fit_transform(df_dummies["Salary_Increased"])

df_dummies["Salary_Diff"] = 1 - LE.fit_transform(df_dummies["Salary_Diff"])
df_dummies["Salary_IncDec"] = 1 - LE.fit_transform(df_dummies["Salary_IncDec"])


display(df_dummies)
print(df_dummies.columns)

,Age,Youth,Black,White,Male,Female,Matric,Tertiary,Pay_Feb,Pay_Apr,Pay_Diff,Salary_Change,Salary_Diff,Salary_Decreased,Salary_Increased,Salary_IncDec,Race_AsInd,Race_Black,Race_Coloured,Race_White,Usual_Work_Clerical_support_workers,Usual_Work_Craft_and_related_trades_workers,Usual_Work_Elementary_occupations,Usual_Work_Managers,Usual_Work_Plant_and_machine_operators_and_assemblers,Usual_Work_Professionals,Usual_Work_Service_and_sales_workers,Usual_Work_Technicians_and_associate_professionals,Province_Eastern_Cape,Province_Free_State,Province_Gauteng,Province_KwaZulu_Natal,Province_Limpopo,Province_Mpumalanga,Province_North_West,Province_Northern_Cape,Province_Western_Cape
15,52.0,0,0,0,0,1,1,0,25000.0,25000.0,0.000000,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
20,40.0,0,1,0,0,1,0,0,3500.0,3500.0,0.000000,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
41,37.0,0,1,0,0,1,1,1,4600.0,4600.0,0.000000,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
44,45.0,0,1,0,0,1,1,1,4000.0,4200.0,5.000000,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
45,39.0,0,1,0,0,1,0,1,3500.0,7200.0,105.714286,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6982,35.0,0,1,0,0,1,1,1,5200.0,5300.0,1.923077,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
7001,43.0,0,1,0,0,1,0,0,3200.0,3200.0,0.000000,0,-1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7016,29.0,0,1,0,1,0,0,0,4400.0,4500.0,2.272727,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
7018,30.0,0,1,0,0,1,1,1,24000.0,12000.0,-50.000000,1,1,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


Index(['Age', 'Youth', 'Black', 'White', 'Male', 'Female', 'Matric',
       'Tertiary', 'Pay_Feb', 'Pay_Apr', 'Pay_Diff', 'Salary_Change',
       'Salary_Diff', 'Salary_Decreased', 'Salary_Increased', 'Salary_IncDec',
       'Race_AsInd', 'Race_Black', 'Race_Coloured', 'Race_White',
       'Usual_Work_Clerical_support_workers',
       'Usual_Work_Craft_and_related_trades_workers',
       'Usual_Work_Elementary_occupations', 'Usual_Work_Managers',
       'Usual_Work_Plant_and_machine_operators_and_assemblers',
       'Usual_Work_Professionals', 'Usual_Work_Service_and_sales_workers',
       'Usual_Work_Technicians_and_associate_professionals',
       'Province_Eastern_Cape', 'Province_Free_State', 'Province_Gauteng',
       'Province_KwaZulu_Natal', 'Province_Limpopo', 'Province_Mpumalanga',
       'Province_North_West', 'Province_Northern_Cape',
       'Province_Western_Cape'],
      dtype='object')


# LOGISTIC 

In [531]:
fuk = smf.logit(formula = "Salary_Change ~ Tertiary", data=df_dummies).fit()
fuk.summary()

Optimization terminated successfully.
         Current function value: 0.661020
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Salary_Change   No. Observations:                  668
Model:                          Logit   Df Residuals:                      666
Method:                           MLE   Df Model:                            1
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:                0.007973
Time:                        19:08:24   Log-Likelihood:                -441.56
converged:                       True   LL-Null:                       -445.11
Covariance Type:            nonrobust   LLR p-value:                  0.007718
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2634      0.110     -2.393      0.017      -0.479      -0.048
Tertiary      -0.4252      0.160     -2.655      0.008      -0.739      -0.111
==============================================================================
"""

In [532]:
fuk = smf.logit(formula = "Salary_Increased ~ Tertiary", data=df_dummies).fit()
fuk.summary()

Optimization terminated successfully.
         Current function value: 0.436495
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       Salary_Increased   No. Observations:                  668
Model:                          Logit   Df Residuals:                      666
Method:                           MLE   Df Model:                            1
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:                0.007893
Time:                        19:08:24   Log-Likelihood:                -291.58
converged:                       True   LL-Null:                       -293.90
Covariance Type:            nonrobust   LLR p-value:                   0.03124
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4469      0.139    -10.415      0.000      -1.719      -1.175
Tertiary      -0.4583      0.215     -2.136      0.033      -0.879      -0.038
==============================================================================
"""

In [545]:
a = smf.logit(formula = "Salary_Increased ~ Race_White + Race_Black", data=df_dummies).fit()
a.summary()

Optimization terminated successfully.
         Current function value: 0.431711
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       Salary_Increased   No. Observations:                  668
Model:                          Logit   Df Residuals:                      665
Method:                           MLE   Df Model:                            2
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:                 0.01877
Time:                        19:11:57   Log-Likelihood:                -288.38
converged:                       True   LL-Null:                       -293.90
Covariance Type:            nonrobust   LLR p-value:                  0.004023
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.1861      0.334     -6.554      0.000      -2.840      -1.532
Race_White    -0.4386      0.571     -0.769      0.442      -1.557       0.680
Race_Black     0.7089      0.353      2.008      0.045       0.017       1.401
==============================================================================
"""

In [485]:
a = smf.logit(formula = "Salary_Change ~ Black", data=df_dummies).fit()
a.summary()

Optimization terminated successfully.
         Current function value: 0.532103
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Salary_Change   No. Observations:                  668
Model:                          Logit   Df Residuals:                      666
Method:                           MLE   Df Model:                            1
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:               0.0009471
Time:                        18:53:20   Log-Likelihood:                -355.45
converged:                       True   LL-Null:                       -355.78
Covariance Type:            nonrobust   LLR p-value:                    0.4117
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3719      0.189     -7.249      0.000      -1.743      -1.001
Black          0.1767      0.217      0.814      0.416      -0.249       0.602
==============================================================================
"""

In [533]:
argh = smf.logit(formula = "Salary_Increased ~  Usual_Work_Craft_and_related_trades_workers + Usual_Work_Clerical_support_workers + Usual_Work_Plant_and_machine_operators_and_assemblers + Usual_Work_Professionals + Usual_Work_Service_and_sales_workers + Usual_Work_Technicians_and_associate_professionals", data=df_dummies).fit()
argh.summary()

Optimization terminated successfully.
         Current function value: 0.421480
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       Salary_Increased   No. Observations:                  668
Model:                          Logit   Df Residuals:                      661
Method:                           MLE   Df Model:                            6
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:                 0.04202
Time:                        19:08:38   Log-Likelihood:                -281.55
converged:                       True   LL-Null:                       -293.90
Covariance Type:            nonrobust   LLR p-value:                 0.0003882
=========================================================================================================================
                                                            coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                                -1.8827      0.234     -8.038      0.000      -2.342      -1.424
Usual_Work_Craft_and_related_trades_workers               0.6931      0.491      1.411      0.158      -0.269       1.656
Usual_Work_Clerical_support_workers                       0.1050      0.390      0.269      0.788      -0.660       0.870
Usual_Work_Plant_and_machine_operators_and_assemblers     0.6300      0.428      1.471      0.141      -0.209       1.469
Usual_Work_Professionals                                 -0.1069      0.355     -0.301      0.763      -0.802       0.589
Usual_Work_Service_and_sales_workers                      0.8382      0.301      2.787      0.005       0.249       1.428
Usual_Work_Technicians_and_associate_professionals       -1.2528      0.635     -1.974      0.048      -2.497      -0.009
=========================================================================================================================
"""

In [534]:
argh = smf.logit(formula = "Salary_Increased ~ Usual_Work_Service_and_sales_workers", data=df_dummies).fit()
argh.summary()

Optimization terminated successfully.
         Current function value: 0.430476
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       Salary_Increased   No. Observations:                  668
Model:                          Logit   Df Residuals:                      666
Method:                           MLE   Df Model:                            1
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:                 0.02157
Time:                        19:08:47   Log-Likelihood:                -287.56
converged:                       True   LL-Null:                       -293.90
Covariance Type:            nonrobust   LLR p-value:                 0.0003695
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -1.8818      0.129    -14.562      0.000      -2.135      -1.629
Usual_Work_Service_and_sales_workers     0.8372      0.229      3.662      0.000       0.389       1.285
========================================================================================================
"""

In [576]:
a = smf.logit(formula = "Salary_Increased ~ Province_Free_State + Province_Gauteng + Province_KwaZulu_Natal + Province_Limpopo + Province_Mpumalanga + Province_North_West + Province_Northern_Cape", data=df_dummies).fit()
a.summary()

Optimization terminated successfully.
         Current function value: 0.431902
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       Salary_Increased   No. Observations:                  668
Model:                          Logit   Df Residuals:                      660
Method:                           MLE   Df Model:                            7
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:                 0.01833
Time:                        19:18:48   Log-Likelihood:                -288.51
converged:                       True   LL-Null:                       -293.90
Covariance Type:            nonrobust   LLR p-value:                    0.1487
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -2.0711      0.274     -7.559      0.000      -2.608      -1.534
Province_Free_State        0.7101      0.448      1.585      0.113      -0.168       1.588
Province_Gauteng           0.5512      0.344      1.601      0.109      -0.124       1.226
Province_KwaZulu_Natal     0.8026      0.350      2.293      0.022       0.117       1.489
Province_Limpopo          -0.1802      0.593     -0.304      0.761      -1.342       0.982
Province_Mpumalanga        0.5512      0.403      1.369      0.171      -0.238       1.340
Province_North_West        0.1615      0.602      0.268      0.788      -1.018       1.341
Province_Northern_Cape    -0.2315      0.543     -0.426      0.670      -1.296       0.833
==========================================================================================
"""

In [579]:
a = smf.logit(formula = "Salary_Increased ~ Black + Tertiary + Province_KwaZulu_Natal", data=df_dummies).fit()
a.summary()

Optimization terminated successfully.
         Current function value: 0.428843
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       Salary_Increased   No. Observations:                  668
Model:                          Logit   Df Residuals:                      664
Method:                           MLE   Df Model:                            3
Date:                Sat, 19 Jun 2021   Pseudo R-squ.:                 0.02528
Time:                        19:19:23   Log-Likelihood:                -286.47
converged:                       True   LL-Null:                       -293.90
Covariance Type:            nonrobust   LLR p-value:                  0.001938
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -2.1680      0.299     -7.251      0.000      -2.754      -1.582
Black                      0.7555      0.301      2.514      0.012       0.167       1.345
Tertiary                  -0.3473      0.218     -1.591      0.112      -0.775       0.081
Province_KwaZulu_Natal     0.3296      0.255      1.295      0.195      -0.169       0.828
==========================================================================================
"""

# LINEAR

In [310]:
a = smf.ols(formula = "Pay_Diff ~ Province_Eastern_Cape + Province_Free_State + Province_Gauteng + Province_KwaZulu_Natal + Province_Limpopo + Province_Mpumalanga + Province_North_West + Province_Northern_Cape + Province_Western_Cape", data=df_dummies).fit()
a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.112
Date:                Sat, 19 Jun 2021   Prob (F-statistic):              0.353
Time:                        18:14:50   Log-Likelihood:                 456.14
No. Observations:                 668   AIC:                            -894.3
Df Residuals:                     659   BIC:                            -853.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.3580      0.005     74.629      0.000       0.349       0.367
Province_Eastern_Cape      0.0533      0.015      3.659      0.000       0.025       0.082
Province_Free_State        0.0079      0.016      0.479      0.632      -0.024       0.040
Province_Gauteng           0.0410      0.010      4.088      0.000       0.021       0.061
Province_KwaZulu_Natal     0.0543      0.011      4.923      0.000       0.033       0.076
Province_Limpopo           0.0321      0.018      1.820      0.069      -0.003       0.067
Province_Mpumalanga        0.0528      0.013      3.957      0.000       0.027       0.079
Province_North_West        0.0172      0.020      0.843      0.399      -0.023       0.057
Province_Northern_Cape     0.0404      0.016      2.589      0.010       0.010       0.071
Province_Western_Cape      0.0590      0.014      4.215      0.000       0.032       0.087
==============================================================================
Omnibus:                      122.174   Durbin-Watson:                   1.870
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              979.437
Skew:                          -0.560   Prob(JB):                    2.08e-213
Kurtosis:                       8.826   Cond. No.                     4.25e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.24e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [313]:
firstFucker = smf.ols(formula = "Pay_Diff ~  Usual_Work_Craft_and_related_trades_workers + Usual_Work_Clerical_support_workers + Usual_Work_Plant_and_machine_operators_and_assemblers + Usual_Work_Professionals + Usual_Work_Service_and_sales_workers + Usual_Work_Technicians_and_associate_professionals", data=df_dummies).fit()
firstFucker.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.205
Date:                Sat, 19 Jun 2021   Prob (F-statistic):              0.302
Time:                        18:15:02   Log-Likelihood:                 455.30
No. Observations:                 668   AIC:                            -896.6
Df Residuals:                     661   BIC:                            -865.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================
                                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                                 0.3992      0.010     40.911      0.000       0.380       0.418
Usual_Work_Craft_and_related_trades_workers               0.0094      0.024      0.386      0.700      -0.039       0.058
Usual_Work_Clerical_support_workers                      -0.0103      0.017     -0.619      0.536      -0.043       0.022
Usual_Work_Plant_and_machine_operators_and_assemblers  4.523e-05      0.021      0.002      0.998      -0.041       0.041
Usual_Work_Professionals                                  0.0077      0.014      0.532      0.595      -0.021       0.036
Usual_Work_Service_and_sales_workers                      0.0198      0.014      1.404      0.161      -0.008       0.047
Usual_Work_Technicians_and_associate_professionals       -0.0227      0.017     -1.299      0.194      -0.057       0.012
==============================================================================
Omnibus:                      130.527   Durbin-Watson:                   1.873
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              930.831
Skew:                          -0.662   Prob(JB):                    7.46e-203
Kurtosis:                       8.630   Cond. No.                         6.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [314]:
second = smf.ols(formula = "Pay_Diff ~ Male", data=df_dummies).fit()
second.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     4.254
Date:                Sat, 19 Jun 2021   Prob (F-statistic):             0.0395
Time:                        18:15:02   Log-Likelihood:                 453.79
No. Observations:                 668   AIC:                            -903.6
Df Residuals:                     666   BIC:                            -894.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3921      0.007     58.854      0.000       0.379       0.405
Male           0.0196      0.010      2.063      0.040       0.001       0.038
==============================================================================
Omnibus:                      123.923   Durbin-Watson:                   1.868
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              996.733
Skew:                          -0.572   Prob(JB):                    3.65e-217
Kurtosis:                       8.874   Cond. No.                         2.60
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [316]:
third = smf.ols(formula = "Pay_Diff ~ Usual_Work_Service_and_sales_workers", data=df_dummies).fit()
third.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     3.678
Date:                Sat, 19 Jun 2021   Prob (F-statistic):             0.0556
Time:                        18:15:21   Log-Likelihood:                 453.51
No. Observations:                 668   AIC:                            -903.0
Df Residuals:                     666   BIC:                            -894.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                0.3969      0.005     73.786      0.000       0.386       0.407
Usual_Work_Service_and_sales_workers     0.0221      0.012      1.918      0.056      -0.001       0.045
==============================================================================
Omnibus:                      130.829   Durbin-Watson:                   1.870
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              963.172
Skew:                          -0.653   Prob(JB):                    7.08e-210
Kurtosis:                       8.736   Cond. No.                         2.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [317]:
fourth = smf.ols(formula = "Pay_Diff ~ Youth", data=df_dummies).fit()
fourth.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.1214
Date:                Sat, 19 Jun 2021   Prob (F-statistic):              0.728
Time:                        18:15:21   Log-Likelihood:                 451.73
No. Observations:                 668   AIC:                            -899.5
Df Residuals:                     666   BIC:                            -890.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4021      0.005     82.229      0.000       0.393       0.412
Youth         -0.0077      0.022     -0.348      0.728      -0.051       0.036
==============================================================================
Omnibus:                      128.856   Durbin-Watson:                   1.870
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1005.374
Skew:                          -0.619   Prob(JB):                    4.85e-219
Kurtosis:                       8.881   Cond. No.                         4.63
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
best = smf.ols(formula = "Pay_Diff ~ Male + Race_White + Usual_Work_Service_and_sales_workers", data=df_dummies).fit()
best.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     3.772
Date:                Sat, 19 Jun 2021   Prob (F-statistic):             0.0120
Time:                        15:47:16   Log-Likelihood:                -770.31
No. Observations:                 158   AIC:                             1549.
Df Residuals:                     154   BIC:                             1561.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                              -50.8323      3.830    -13.273      0.000     -58.398     -43.266
Male                                     9.2784      5.184      1.790      0.075      -0.964      19.520
Race_White                              -2.5722      7.754     -0.332      0.741     -17.891      12.746
Usual_Work_Service_and_sales_workers    15.0086      5.974      2.513      0.013       3.208      26.809
==============================================================================
Omnibus:                       20.359   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.812
Skew:                          -0.557   Prob(JB):                      0.00165
Kurtosis:                       2.159   Cond. No.                         3.63
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
best = smf.ols(formula = "Pay_Diff ~ Male + White ", data=df_dummies).fit()
best.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     2.419
Date:                Sat, 19 Jun 2021   Prob (F-statistic):             0.0923
Time:                        15:50:58   Log-Likelihood:                -773.48
No. Observations:                 158   AIC:                             1553.
Df Residuals:                     155   BIC:                             1562.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -47.4141      3.641    -13.023      0.000     -54.606     -40.222
Male          10.8691      5.233      2.077      0.039       0.532      21.207
White         -5.1258      7.818     -0.656      0.513     -20.569      10.318
==============================================================================
Omnibus:                       22.515   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.532
Skew:                          -0.528   Prob(JB):                      0.00190
Kurtosis:                       2.113   Cond. No.                         3.43
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""